In [ ]:
# Add the relevant scripts from LArMachineLearningData
# Nice the process so it can run with lots of cores on low priority
import os
os.nice(20)

# Add local paths
import sys
hnlDIR = os.environ['_']
sys.path.append('../pyscript')

# From pyscript Library
from Plotting import *
from Dictionary import *
from HelperFunctions import *
from CutFunctions import *

import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

<h1>Configuration Stuff Here</h1>

In [ ]:
ifTune = False
ifSave = False
savePath = "../plot_files/06Feb24/"

In [ ]:
ifStrict = True

In [ ]:
TuneCut = {}

if ifStrict:   
    TuneCut = {
    'crumbs': 0.04
    , 'Opt0Score': 1000
    ,'Opt0FracMore': -0.02
    , 'Opt0FracLess': 0.92
    , 'Muon' : 0.02
    , 'Proton' : 0.02
    , 'Pion' : 0.02
    , 'ShowerTheta': 9.2
    }
    
else:
    TuneCut = {
    'crumbs': 0.04
    , 'Opt0Score': 1000
    , 'Opt0FracMore': -0.08
    , 'Opt0FracLess': 0.92
    , 'Muon' : 0.06
    , 'Proton' : 1
    , 'Pion' : 0.98
    , 'ShowerTheta': 12.8
    }
    

<h1> Read in PKL Dataframe </h1>

<h3> HNL </h3>

In [ ]:
df_hnl = pd.DataFrame()

for i in range(0, 2):
    hnl_path = "../pkl_files/v1/hnl_m200_50k__slc_{}.pkl".format(i)
    print(hnl_path)
    
    file = open(hnl_path, 'rb')
    df = pickle.load(file)
    
    df_hnl = pd.concat((df_hnl, df), ignore_index=True)
    file.close()
    
#--------------------------------------------------#
dfsubrun_hnl = pd.DataFrame()

for i in range(0, 2):
    hnl_path = "../pkl_files/v1/hnl_m200_50k__subrun_{}.pkl".format(i)
    print(hnl_path)
    
    file = open(hnl_path, 'rb')
    df = pickle.load(file)
    
    dfsubrun_hnl = pd.concat((dfsubrun_hnl, df), ignore_index=True)
    file.close()

<h3> TPC Neutrino </h3>

In [ ]:
df_nu = pd.DataFrame()

for i in range(0, 4):
    nu_path = "../pkl_files/v1/nu_tpc_100k_slc_{}.pkl".format(i)
    print(nu_path)
    
    file = open(nu_path, 'rb')
    df = pickle.load(file)
    
    df_nu = pd.concat((df_nu, df), ignore_index=True)
    file.close()
    
#--------------------------------------------------#
dfsubrun_nu = pd.DataFrame()

for i in range(0, 4):
    nu_path = "../pkl_files/v1/nu_tpc_100k_subrun_{}.pkl".format(i)
    print(nu_path)
    
    file = open(nu_path, 'rb')
    df = pickle.load(file)
    
    dfsubrun_nu = pd.concat((dfsubrun_nu, df), ignore_index=True)
    file.close()

<h3> Intime Cosmics </h3>

In [ ]:
df_cos = pd.DataFrame()

for i in range(0, 2):
    cos_path = "../pkl_files/v1/cos_50k_slc_{}.pkl".format(i)
    print(cos_path)
    
    file = open(cos_path, 'rb')
    df = pickle.load(file)
    
    df_cos = pd.concat((df_cos, df), ignore_index=True)
    file.close()
    
#--------------------------------------------------#
dfsubrun_cos = pd.DataFrame()

for i in range(0, 2):
    cos_path = "../pkl_files/v1/cos_50k_subrun_{}.pkl".format(i)
    print(cos_path)
    
    file = open(cos_path, 'rb')
    df = pickle.load(file)
    
    dfsubrun_cos = pd.concat((dfsubrun_cos, df), ignore_index=True)
    file.close()

<h1>Some Scaling Voodoo</h1>

<h3>HNL</h3>

In [ ]:
#Copy from text file that contains slice count before clear cosmics cut

true_signals = 17442 + 17207
true_nonfv_signals = 7608 + 7443
total_true_signals = 25050 + 24650
start_signals = 16653 + 16435
start_nonfv_signals = 6370 + 6261
total_start_signals = 23023 + 22696

In [ ]:
#Sanity Check: ifScale == 1 i.e. Umu = 1e-7 and 3 years POT
#true counts ~1.5k
#start counts ~1.4k
scale_pot_hnl, hnl_spill = calc_scaling_pot(dfsubrun_hnl, df_hnl, ifScale = 1)

true_signals = true_signals * scale_pot_hnl
true_nonfv_signals = true_nonfv_signals * scale_pot_hnl
total_true_signals = total_true_signals * scale_pot_hnl
start_signals = start_signals * scale_pot_hnl
start_nonfv_signals = start_nonfv_signals * scale_pot_hnl
total_start_signals = total_start_signals * scale_pot_hnl

true_counts = total_true_signals
start_counts = total_start_signals
print("true counts = " + str(true_counts))
print("start counts = " + str(start_counts))

In [ ]:
simU = 1e-7 #already squared
scaleHNLPlot = 80

In [ ]:
plotU = getUfromScaleFactor(simU, scaleHNLPlot)
true_counts = true_counts * scaleHNLPlot
start_counts =  start_counts * scaleHNLPlot

print("Scale Umu by " + str(scaleHNLPlot))
print("Scaled Umu = " + str(plotU))
print("Scaled true counts = " + str(true_counts))
print("Scaled start counts = " + str(start_counts))

In [ ]:
scale_pot_hnl, hnl_spill = calc_scaling_pot(dfsubrun_hnl, df_hnl, ifScale = scaleHNLPlot)

del dfsubrun_hnl

<h3>TPC Neutrino</h3>

In [ ]:
scale_pot_nu, nu_spill = calc_scaling_pot(dfsubrun_nu, df_nu)

del  dfsubrun_nu

<h3>Intime Cosmics</h3>

In [ ]:
scale_pot_cosmics = calc_scaling_spill(dfsubrun_cos, df_cos, hnl_spill, nu_spill)

del  dfsubrun_cos

<h1> Make Beam Bucket - Pre Cosmics </h1>

In [ ]:
width = 19.02

df_hnl['mod_t'] = df_hnl['slc_opt0_time_corrected_Z_pandora']%width
df_nu['mod_t'] = df_nu['slc_opt0_time_corrected_Z_pandora']%width
df_cos['mod_t'] = df_cos['slc_opt0_time_corrected_Z_pandora']%width
#----------------------------------------------
hist, bins = plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'mod_t', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 19, xnbin = 19,
                    xtitle = 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"beam_bucket_precosmics.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1> Save Purity and Eff </h1>

In [ ]:
purity_arr, eff_arr, cut_arr = [], [], []

In [ ]:
purity, eff = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)


purity_arr.append(purity)
eff_arr.append(eff)
cut_arr.append("Clear Cosmics Removal")

print(purity_arr)
print(eff_arr)
print(cut_arr)

<h1>Beam Gate</h1>

<h3>Apply Cut</h3>

<h3>Save Purity/Eff</h3>

<h1> CRUMBS </h1>

In [ ]:
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_crumbs_score', 
                    scaleHNLPlot,
                    xmin = -2.5, xmax =1, xnbin = 40,
                    xtitle = 'CRUMBS Score'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"crumbs_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

In [ ]:
def vary_crumbsScore(df_hnl, df_nu, df_cos):
    
    step = 0.02
    cutStep = np.arange(-1.0, 1 + step, step)
    
    p_arr, eff_arr, peff_arr = [], [], []
    
    purity_start, eff_start = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)
    
    for c in cutStep:
        df_hnl_cut = cutCosmics(df_hnl, crumbsScore = c)
        df_nu_cut = cutCosmics(df_nu, crumbsScore = c)
        df_cos_cut = cutCosmics(df_cos, crumbsScore = c)
        
        purity, eff = calc_purity_eff(df_hnl_cut, df_nu_cut, df_cos_cut, true_counts, start_counts)
        
        p_arr.append(purity)
        eff_arr.append(eff)
        peff_arr.append(purity*eff)
    
    plot_purity_eff(p_arr, eff_arr, peff_arr, cutStep, purity_start, eff_start, loc = 'center left')


In [ ]:
if ifTune:
    vary_crumbsScore(df_hnl, df_nu, df_cos)
    
    if ifSave:
        plt.savefig(savePath+"/crumbs_tuning.png", dpi =200)

<h3>Apply Cut</h3>

In [ ]:
score = TuneCut['crumbs']

df_hnl = cutCosmics(df_hnl, crumbsScore = score)
df_nu = cutCosmics(df_nu, crumbsScore = score)
df_cos = cutCosmics(df_cos, crumbsScore = score)

<h3>Save Purity/Eff</h3>

In [ ]:
purity, eff = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)

purity_arr.append(purity)
eff_arr.append(eff)
cut_arr.append("CRUMBS")

print(purity_arr)
print(eff_arr)
print(cut_arr)

In [ ]:
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_crumbs_score', 
                    scaleHNLPlot,
                    xmin = -2.5, xmax =1, xnbin = 40,
                    xtitle = 'CRUMBS Score'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"crumbs_postcut.png", dpi = 200)
#-----------------------------------#

plt.show()

<h1> Fiducial Volume </h1>

In [ ]:
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_is_fv', 
                    scaleHNLPlot,
                    xmin = -1, xmax = 3, xnbin = 40,
                    xtitle = 'Vertex is in Fiducial Volume'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"fv_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

<h3>Apply Cut</h3>

In [ ]:
df_hnl = cutFV(df_hnl)
df_nu = cutFV(df_nu)
df_cos = cutFV(df_cos)

<h3>Save Purity/Eff</h3>

In [ ]:
purity, eff = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)

purity_arr.append(purity)
eff_arr.append(eff)
cut_arr.append("Vertex In FV")

print(purity_arr)
print(eff_arr)
print(cut_arr)

In [ ]:
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_is_fv', 
                    scaleHNLPlot,
                    xmin = -1, xmax = 3, xnbin = 40,
                    xtitle = 'Vertex is in Fiducial Volume'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"fv_postcut.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1> Opt0 Score </h1>

In [ ]:
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_opt0_score', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 40000, xnbin = 40,
                    xtitle = 'Opt0 Score'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"opt0score_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

In [ ]:
def vary_OpT0Score(df_hnl, df_nu, df_cos):
    
    step = 50
    cutStep = np.arange(0, 2000 + step, step)
    
    p_arr, eff_arr, peff_arr = [], [], []
    
    purity_start, eff_start = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)
  
    for c in cutStep:
        df_hnl_cut = cutOpt0Score(df_hnl, Opt0Score = c)
        df_nu_cut = cutOpt0Score(df_nu, Opt0Score = c)
        df_cos_cut = cutOpt0Score(df_cos, Opt0Score = c)
        
        purity, eff = calc_purity_eff(df_hnl_cut, df_nu_cut, df_cos_cut, true_counts, start_counts)
        
        p_arr.append(purity)
        eff_arr.append(eff)
        peff_arr.append(purity*eff)
     
    plot_purity_eff(p_arr, eff_arr, peff_arr, cutStep, purity_start, eff_start, loc = 'center left')

In [ ]:
if ifTune:
    vary_OpT0Score(df_hnl, df_nu, df_cos)
    if ifSave:
        plt.savefig(savePath+"/opt0score_tuning.png", dpi =200)

<h3> Apply Cut </h3>

In [ ]:
score = TuneCut['Opt0Score']

df_hnl = cutOpt0Score(df_hnl, Opt0Score = score)
df_nu = cutOpt0Score(df_nu, Opt0Score = score)
df_cos = cutOpt0Score(df_cos, Opt0Score = score)

<h3>Save Purity/Eff</h3>

In [ ]:
purity, eff = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)

purity_arr.append(purity)
eff_arr.append(eff)
cut_arr.append("Opt0 Score")

print(purity_arr)
print(eff_arr)
print(cut_arr)

In [ ]:
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_opt0_score', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 40000, xnbin = 40,
                    xtitle = 'Opt0 Score'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"opt0score_postcut.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1>Opt0 Frac </h1>

In [ ]:
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_opt0_frac', 
                    scaleHNLPlot,
                    xmin = -1, xmax = 2, xnbin = 40,
                    xtitle = 'Opt0 Fraction'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"opt0frac_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

In [ ]:
def vary_OpT0FracMore(df_hnl, df_nu, df_cos):
    
    step = 0.02
    cutStep = np.arange(-1, 0.5 + step, step)
    
    p_arr, eff_arr, peff_arr = [], [], []
    
    purity_start, eff_start = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)
    
    for c in cutStep:
        df_hnl_cut = cutOpt0Frac(df_hnl, Opt0FracMore = c, Opt0FracLess = 999)
        df_nu_cut = cutOpt0Frac(df_nu, Opt0FracMore = c, Opt0FracLess = 999)
        df_cos_cut = cutOpt0Frac(df_cos, Opt0FracMore = c, Opt0FracLess = 999)
        
        purity, eff = calc_purity_eff(df_hnl_cut, df_nu_cut, df_cos_cut, true_counts, start_counts)
    
        p_arr.append(purity)
        eff_arr.append(eff)
        peff_arr.append(purity*eff)
     
    plot_purity_eff(p_arr, eff_arr, peff_arr, cutStep, purity_start, eff_start, loc = 'center left')

In [ ]:
def vary_OpT0FracLess(df_hnl, df_nu, df_cos):
    
    step = 0.02
    cutStep = np.arange(0, 2 + step, step)
    
    p_arr, eff_arr, peff_arr = [], [], []
    
    purity_start, eff_start = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)
    
    for c in cutStep:
        df_hnl_cut = cutOpt0Frac(df_hnl, Opt0FracMore = -999, Opt0FracLess = c)
        df_nu_cut = cutOpt0Frac(df_nu, Opt0FracMore = -999, Opt0FracLess = c)
        df_cos_cut = cutOpt0Frac(df_cos, Opt0FracMore = -999, Opt0FracLess = c)
        
        purity, eff = calc_purity_eff(df_hnl_cut, df_nu_cut, df_cos_cut, true_counts, start_counts)
    
        p_arr.append(purity)
        eff_arr.append(eff)
        peff_arr.append(purity*eff)
     
    plot_purity_eff(p_arr, eff_arr, peff_arr, cutStep, purity_start, eff_start, loc = 'center right')

In [ ]:
if ifTune:
    vary_OpT0FracMore(df_hnl, df_nu, df_cos)
    if ifSave:
        plt.savefig(savePath+"/opt0frac_more_tuning.png", dpi =200)

In [ ]:
if ifTune:
    vary_OpT0FracLess(df_hnl, df_nu, df_cos)
    if ifSave:
        plt.savefig(savePath+"/opt0frac_less_tuning.png", dpi =200)

<h3> Apply Cut </h3>

In [ ]:
fracMore = TuneCut['Opt0FracMore']
fracLess = TuneCut['Opt0FracLess']

df_hnl = cutOpt0Frac(df_hnl, Opt0FracMore = fracMore, Opt0FracLess = fracLess)
df_nu = cutOpt0Frac(df_nu, Opt0FracMore = fracMore, Opt0FracLess = fracLess)
df_cos = cutOpt0Frac(df_cos, Opt0FracMore = fracMore, Opt0FracLess = fracLess)

<h3>Save Purity/Eff</h3>

In [ ]:
purity, eff = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)

purity_arr.append(purity)
eff_arr.append(eff)
cut_arr.append("Opt0 Fraction")

print(purity_arr)
print(eff_arr)
print(cut_arr)

In [ ]:
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_opt0_frac', 
                    scaleHNLPlot,
                    xmin = -1, xmax = 2, xnbin = 40,
                    xtitle = 'Opt0 Fraction'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"opt0frac_postcut.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1> Make Beam Bucket - Pre PID</h1>

In [ ]:
hist, bins = plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'mod_t', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 19, xnbin = 19,
                    xtitle = 'Opt0 Time Corrected Z % ' + str(width) + " [ns]"
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"beam_bucket_postcosmics_prepid.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1>Muon Rejection</h1>

In [ ]:
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_n_razzled_muons', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 5, xnbin = 5,
                    xtitle = 'n Razzled Muons'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"nrazzled_muon_precut.png", dpi = 200)
#-----------------------------------#
plt.show()
#----------------------------------------------
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_razzled_muon_score', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 1, xnbin = 40,
                    xtitle = 'PFP Razzled Muon Score'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"razzled_muon_score_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

In [ ]:
def vary_MuonScore(df_hnl, df_nu, df_cos):
    
    step = 0.02
    cutStep = np.arange(0, 1 + step, step)
    
    p_arr, eff_arr, peff_arr = [], [], []
    
    purity_start, eff_start = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)
    
    for c in cutStep:
        df_hnl_cut = cutMuon(df_hnl, muonScore = c)
        df_nu_cut = cutMuon(df_nu, muonScore = c)
        df_cos_cut = cutMuon(df_cos, muonScore = c)
        
        purity, eff = calc_purity_eff(df_hnl_cut, df_nu_cut, df_cos_cut, true_counts, start_counts)
        
        p_arr.append(purity)
        eff_arr.append(eff)
        peff_arr.append(purity*eff)
     
    plot_purity_eff(p_arr, eff_arr, peff_arr, cutStep, purity_start, eff_start , loc = 'center right')


In [ ]:
if ifTune:
    vary_MuonScore(df_hnl, df_nu, df_cos)
    if ifSave:
        plt.savefig(savePath+"/muon_tuning.png", dpi =200)

<h3> Apply Cut</h3>

In [ ]:
muonScore = TuneCut['Muon']

df_hnl = cutMuon(df_hnl, muonScore =muonScore)
df_nu = cutMuon(df_nu, muonScore =muonScore)
df_cos = cutMuon(df_cos,muonScore =muonScore)

<h3>Save Purity/Eff</h3>

In [ ]:
purity, eff = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)

purity_arr.append(purity)
eff_arr.append(eff)
cut_arr.append("Muon Cut")

print(purity_arr)
print(eff_arr)
print(cut_arr)

In [ ]:
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_n_razzled_muons', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 5, xnbin = 5,
                    xtitle = 'n Razzled Muons'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"nrazzled_muon_postcut.png", dpi = 200)
#-----------------------------------#
plt.show()
#----------------------------------------------
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_razzled_muon_score', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 1, xnbin = 40,
                    xtitle = 'PFP Razzled Muon Score'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"razzled_muon_score_postcut.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1>Proton Rejection</h1>

In [ ]:
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_n_razzled_protons_thresh', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 5, xnbin = 5,
                    xtitle = 'n Razzled Protons'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"nrazzled_proton_precut.png", dpi = 200)
#-----------------------------------#
plt.show()
#----------------------------------------------
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_razzled_proton_score', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 1, xnbin = 40,
                    xtitle = 'PFP Razzled Proton Score'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"razzled_proton_score_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

In [ ]:
def vary_ProtonScore(df_hnl, df_nu, df_cos):
    
    step = 0.02
    cutStep = np.arange(0, 1 + step, step)
    
    p_arr, eff_arr, peff_arr = [], [], []
    
    purity_start, eff_start  = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)
    
    for c in cutStep:
        df_hnl_cut = cutProton(df_hnl, nProton = 0, protonScore = c)
        df_nu_cut = cutProton(df_nu, nProton = 0, protonScore = c)
        df_cos_cut = cutProton(df_cos, nProton = 0, protonScore = c)
        
        purity, eff = calc_purity_eff(df_hnl_cut, df_nu_cut, df_cos_cut, true_counts, start_counts)
        
        p_arr.append(purity)
        eff_arr.append(eff)
        peff_arr.append(purity*eff)
     
    plot_purity_eff(p_arr, eff_arr, peff_arr, cutStep, purity_start, eff_start, loc = 'center right')
    #plt.xlim(0, 0.1)

In [ ]:
if ifTune:
    vary_ProtonScore(df_hnl, df_nu, df_cos)
    if ifSave:
        plt.savefig(savePath+"/proton_tuning.png", dpi =200)

<h3>Apply Cut</h3>

In [ ]:
protonScore = TuneCut['Proton']

df_hnl = cutProton(df_hnl, nProton = 0, protonScore = protonScore)
df_nu = cutProton(df_nu, nProton = 0, protonScore = protonScore)
df_cos = cutProton(df_cos, nProton = 0, protonScore = protonScore)

<h3>Save Purity/Eff</h3>

In [ ]:
purity, eff = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)

purity_arr.append(purity)
eff_arr.append(eff)
cut_arr.append("Proton Cut")

print(purity_arr)
print(eff_arr)
print(cut_arr)

In [ ]:
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_n_razzled_protons_thresh', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 5, xnbin = 5,
                    xtitle = 'n Razzled Protons'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"nrazzled_proton_postcut.png", dpi = 200)
#-----------------------------------#
plt.show()
#----------------------------------------------
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_razzled_proton_score', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 1, xnbin = 40,
                    xtitle = 'PFP Razzled Proton Score'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"razzled_proton_score_postcut.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1>Pion Rejection</h1>

In [ ]:
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_n_razzled_pions_thresh', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 5, xnbin = 5,
                    xtitle = 'n Razzled Pions'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"nrazzled_pion_precut.png", dpi = 200)
#-----------------------------------#
plt.show()
#----------------------------------------------
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_razzled_pion_score', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 1, xnbin = 40,
                    xtitle = 'PFP Razzled Pion Score'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"razzled_pion_score_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

In [ ]:
def vary_PionScore(df_hnl, df_nu, df_cos):
    
    step = 0.02
    cutStep = np.arange(0, 1 + step, step)
    
    p_arr, eff_arr, peff_arr = [], [], []
    
    purity_start, eff_start  = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)
    
    for c in cutStep:
        df_hnl_cut = cutPion(df_hnl, nPion = 0, pionScore = c)
        df_nu_cut = cutPion(df_nu, nPion = 0, pionScore = c)
        df_cos_cut = cutPion(df_cos, nPion = 0, pionScore = c)
        
        purity, eff = calc_purity_eff(df_hnl_cut, df_nu_cut, df_cos_cut, true_counts, start_counts)
        
        p_arr.append(purity)
        eff_arr.append(eff)
        peff_arr.append(purity*eff)
     
    plot_purity_eff(p_arr, eff_arr, peff_arr, cutStep, purity_start, eff_start, loc = 'center right')


In [ ]:
if ifTune:
    vary_PionScore(df_hnl, df_nu, df_cos)
    if ifSave:
        plt.savefig(savePath+"/pion_tuning.png", dpi =200)

<h3>Apply Cut </h3>

In [ ]:
pionScore = TuneCut['Pion']

df_hnl = cutPion(df_hnl, nPion = 0, pionScore = pionScore)
df_nu = cutPion(df_nu, nPion = 0, pionScore = pionScore)
df_cos = cutPion(df_cos, nPion = 0, pionScore = pionScore)

<h3>Save Purity/Eff</h3>

In [ ]:
purity, eff = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)

purity_arr.append(purity)
eff_arr.append(eff)
cut_arr.append("Pion Cut")

print(purity_arr)
print(eff_arr)
print(cut_arr)

In [ ]:
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_n_razzled_pions_thresh', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 5, xnbin = 5,
                    xtitle = 'n Razzled Pions'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"nrazzled_pion_postcut.png", dpi = 200)
#-----------------------------------#
plt.show()
#----------------------------------------------
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_razzled_pion_score', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 1, xnbin = 40,
                    xtitle = 'PFP Razzled Pion Score'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"razzled_pion_score_postcut.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1>Electron Rejection (?)</h1>

In [ ]:
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_n_razzled_electrons', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 5, xnbin = 5,
                    xtitle = 'n Razzled Electrons'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"nrazzled_electron_precut.png", dpi = 200)
#-----------------------------------#
plt.show()
#----------------------------------------------
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_razzled_electron_score', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 1, xnbin = 40,
                    xtitle = 'PFP Razzled Electron Score'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"razzled_electron_score_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

In [ ]:
def vary_ElectronScoreLess(df_hnl, df_nu, df_cos):
    
    step = 0.02
    cutStep = np.arange(0, 1 + step, step)
    
    p_arr, eff_arr, peff_arr = [], [], []
    
    purity_start, eff_start  = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)
    
    for c in cutStep:
        df_hnl_cut = cutElectronLess(df_hnl, nElectron = 2, electronScore = c)
        df_nu_cut = cutElectronLess(df_nu, nElectron = 2, electronScore = c)
        df_cos_cut = cutElectronLess(df_cos, nElectron = 2, electronScore = c)
        
        purity, eff = calc_purity_eff(df_hnl_cut, df_nu_cut, df_cos_cut, true_counts, start_counts)
    
        p_arr.append(purity)
        eff_arr.append(eff)
        peff_arr.append(purity*eff)
     
    plot_purity_eff(p_arr, eff_arr, peff_arr, cutStep, purity_start, eff_start)

In [ ]:
if ifTune:
    vary_ElectronScoreLess(df_hnl, df_nu, df_cos)
    if ifSave:
        plt.savefig(savePath+"/electron_less_tuning.png", dpi =200)

In [ ]:
def vary_ElectronScoreMore(df_hnl, df_nu, df_cos):
    
    step = 0.02
    cutStep = np.arange(0.02, 1 + step, step)
    p_arr, eff_arr, peff_arr = [], [], []
    
    purity_start, eff_start  = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)
    
    for c in cutStep:
        df_hnl_cut = cutElectronMore(df_hnl, nElectron = 2, electronScore = c)
        df_nu_cut = cutElectronMore(df_nu, nElectron = 2, electronScore = c)
        df_cos_cut = cutElectronMore(df_cos, nElectron = 2, electronScore = c)
        
        purity, eff = calc_purity_eff(df_hnl_cut, df_nu_cut, df_cos_cut, true_counts, start_counts)
    
        purity = purity/100
        eff = eff/100
        
        p_arr.append(purity)
        eff_arr.append(eff)
        peff_arr.append(purity*eff)
     
    plot_purity_eff(p_arr, eff_arr, peff_arr, cutStep, purity_start, eff_start)

In [ ]:
if ifTune:
    vary_ElectronScoreMore(df_hnl, df_nu, df_cos)
    if ifSave:
        plt.savefig(savePath+"/electron_more_tuning.png", dpi =200)

<h3>Apply Cut</h3>

<h3>Save Purity/Eff</h3>

<h1>Photon Selection</h1>

In [ ]:
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_n_razzled_photons', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 10, xnbin = 10,
                    xtitle = 'n Razzled Photons'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"nrazzled_photon_precut.png", dpi = 200)
#-----------------------------------#
plt.show()
#----------------------------------------------
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_razzled_photon_score', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 1, xnbin = 40,
                    xtitle = 'PFP Razzled Photon Score'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"razzled_photon_score_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

In [ ]:
def vary_PhotonScoreLess(df_hnl, df_nu, df_cos):
    
    step = 0.02
    cutStep = np.arange(0, 1 + step, step)
    
    p_arr, eff_arr, peff_arr = [], [], []
    
    purity_start, eff_start  = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)
    
    for c in cutStep:
        df_hnl_cut = cutPhotonLess(df_hnl, nPhoton = 3, photonScore = c)
        df_nu_cut = cutPhotonLess(df_nu, nPhoton = 3, photonScore = c)
        df_cos_cut = cutPhotonLess(df_cos, nPhoton = 3, photonScore = c)
        
        purity, eff = calc_purity_eff(df_hnl_cut, df_nu_cut, df_cos_cut, true_counts, start_counts)
        
        p_arr.append(purity)
        eff_arr.append(eff)
        peff_arr.append(purity*eff)
     
    plot_purity_eff(p_arr, eff_arr, peff_arr, cutStep, purity_start, eff_start, loc = 'center right')

In [ ]:
if ifTune:
    vary_PhotonScoreLess(df_hnl, df_nu, df_cos)
    if ifSave:
        plt.savefig(savePath+"/photon_less_tuning.png", dpi =200)

In [ ]:
def vary_PhotonScoreMore(df_hnl, df_nu, df_cos):
    
    step = 0.02
    cutStep = np.arange(0, 1 + step, step)
    
    p_arr, eff_arr, peff_arr = [], [], []
    
    purity_start, eff_start  = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)
    
    for c in cutStep:
        df_hnl_cut = cutPhotonMore(df_hnl, nPhoton = 3, photonScore = c)
        df_nu_cut = cutPhotonMore(df_nu, nPhoton = 3, photonScore = c)
        df_cos_cut = cutPhotonMore(df_cos, nPhoton = 3, photonScore = c)
        
        purity, eff = calc_purity_eff(df_hnl_cut, df_nu_cut, df_cos_cut, true_counts, start_counts)
        
        p_arr.append(purity)
        eff_arr.append(eff)
        peff_arr.append(purity*eff)
     
    plot_purity_eff(p_arr, eff_arr, peff_arr, cutStep,purity_start, eff_start, loc = 'center right')

In [ ]:
if ifTune:
    vary_PhotonScoreMore(df_hnl, df_nu, df_cos)
    if ifSave:
        plt.savefig(savePath+"/photon_more_tuning.png", dpi =200)

<h3>Apply Cut</h3>

<h3>Save Purity/Eff</h3>

<h1>Shower Theta</h1>

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12, 4), sharey= True, sharex = True)

xmin, xmax = -180, 180
xnbin = 90

ymin, ymax = 0, 180
ynbin = 90

xlimmin, xlimmax = xmin, xmax
ylimmin, ylimmax = ymin, ymax
#----------------------------------------------
plot_2dhist( df_nu['slc_pfp_shower_phi'], df_nu['slc_pfp_shower_theta'], 
            ax1, fig, 
            xmin, xmax, ymin, ymax, xnbin, ynbin,
            xlimmin, xlimmax, ylimmin, ylimmax,
            weights = df_nu['scale_pot'],
            title = 'Neutrino', xtitle = 'Phi', ytitle= 'Theta'
                )
#----------------------------------------------
plot_2dhist( df_hnl['slc_pfp_shower_phi'], df_hnl['slc_pfp_shower_theta'], 
            ax2, fig, 
            xmin, xmax, ymin, ymax, xnbin, ynbin,
            xlimmin, xlimmax, ylimmin, ylimmax,
            weights = df_hnl['scale_pot'],
            title = 'HNL', xtitle = 'Phi', ytitle= 'Theta'
                )
#----------------------------------------------
plot_2dhist( df_cos['slc_pfp_shower_phi'], df_cos['slc_pfp_shower_theta'], 
            ax3, fig, 
            xmin, xmax, ymin, ymax, xnbin, ynbin,
            xlimmin, xlimmax, ylimmin, ylimmax,
            weights = df_cos['scale_pot'],
            title = 'Cosmics', xtitle = 'Phi', ytitle= 'Theta'
                )
#----------------------------------------------
#plt.ylim(0,40)
plt.show()

In [ ]:
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_shower_theta', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 180, xnbin = 20,
                    xtitle = 'Shower Theta'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"shower_theta_precut.png", dpi = 200)
#-----------------------------------#
plt.show()
#----------------------------------------------
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_shower_phi', 
                    scaleHNLPlot,
                    xmin = -180, xmax = 180, xnbin = 90,
                    xtitle = 'Shower Phi'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"shower_phi_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

In [ ]:
def vary_Theta(df_hnl, df_nu, df_cos):
    
    step = 0.2
    cutStep = np.arange(6, 25 + step, step)
    
    p_arr, eff_arr, peff_arr = [], [], []
    
    purity_start, eff_start = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)
    
    for c in cutStep:
        df_hnl_cut = cutThetaAngle(df_hnl, thetaAngle = c) 
        df_nu_cut = cutThetaAngle(df_nu, thetaAngle = c) 
        df_cos_cut = cutThetaAngle(df_cos, thetaAngle = c) 
        
        purity, eff = calc_purity_eff(df_hnl_cut, df_nu_cut, df_cos_cut, true_counts, start_counts)
        
        p_arr.append(purity)
        eff_arr.append(eff)
        peff_arr.append(purity*eff)
    
    pMax = max(p_arr)
    bestIndex = p_arr.index(pMax)
    
    del p_arr[bestIndex]
    del eff_arr[bestIndex]
    del peff_arr[bestIndex]
    cutStep = np.delete(cutStep, bestIndex)
    
    plot_purity_eff(p_arr, eff_arr, peff_arr, cutStep, purity_start, eff_start, loc = 'center right')

In [ ]:
if ifTune:
    vary_Theta(df_hnl, df_nu, df_cos)
    if ifSave:
        plt.savefig(savePath+"/shower_theta_tuning.png", dpi =200)

In [ ]:
angle = TuneCut['ShowerTheta']

df_hnl = cutThetaAngle(df_hnl, thetaAngle = angle) 
df_nu = cutThetaAngle(df_nu, thetaAngle = angle) 
df_cos = cutThetaAngle(df_cos, thetaAngle = angle) 

<h3>Save Purity/Eff</h3>

In [ ]:
purity, eff = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)

purity_arr.append(purity)
eff_arr.append(eff)
cut_arr.append("Shower Theta Cut")

print(purity_arr)
print(eff_arr)
print(cut_arr)

In [ ]:
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_shower_theta', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 180, xnbin = 20,
                    xtitle = 'Shower Theta'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"shower_theta_postcut.png", dpi = 200)
#-----------------------------------#
plt.show()
#----------------------------------------------
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_shower_phi', 
                    scaleHNLPlot,
                    xmin = -180, xmax = 180, xnbin = 90,
                    xtitle = 'Shower Phi'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"shower_phi_postcut.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1>Shower Properties</h1>

In [ ]:
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_shower_energy', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 2000, xnbin = 100,
                    xtitle = 'Shower Energy [MeV]'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"shower_energy_postpid.png", dpi = 200)
#-----------------------------------#
plt.show()
#----------------------------------------------
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_shower_conv_gap', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 1, xnbin = 40,
                    xtitle = 'Shower Conversion Gap [cm]'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"shower_convgap_postpid.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1>Shower Stuff</h1>

<h3> Split into 1 shower and multi shower case </h3>

In [ ]:
def split_my_df(df):
    temp = df[['run', 'subrun', 'event', 'slc_idx'
                   , 'slc_pfp_shower_energy', 'slc_comp', 'scale_pot', 'slc_true_event_type'
                    , 'slc_pfp_shower_dir_x' , 'slc_pfp_shower_dir_y' , 'slc_pfp_shower_dir_z'
                  ]]
    temp["pfp_idx"] = temp.groupby(['run', 'subrun', 'event', 'slc_idx']).transform("cumcount").add(1) - 1
    temp = temp.set_index(['run', 'subrun', 'event', 'slc_idx']).reset_index()
    
    temp1 = temp.groupby(['run', 'subrun', 'event', 'slc_idx'])['slc_pfp_shower_energy'].apply(list).reset_index()
    temp2 = temp.groupby(['run', 'subrun', 'event', 'slc_idx'])['pfp_idx'].apply(list).reset_index()
    temp3 = temp.groupby(['run', 'subrun', 'event', 'slc_idx'])['slc_pfp_shower_dir_x'].apply(list).reset_index()
    temp4 = temp.groupby(['run', 'subrun', 'event', 'slc_idx'])['slc_pfp_shower_dir_y'].apply(list).reset_index()
    temp5 = temp.groupby(['run', 'subrun', 'event', 'slc_idx'])['slc_pfp_shower_dir_z'].apply(list).reset_index()

    concat = pd.concat([temp1, temp2, temp3, temp4, temp5], axis = 1)
    
    concat = concat.loc[:, ~concat.columns.duplicated()]
    
    concat['n_pfp'] = concat.apply(lambda row: len(row['slc_pfp_shower_energy']), axis=1)
    
    #1 shower selection
    shw1 = concat[concat['n_pfp'] == 1]
    shw1 = shw1.drop(columns = ['slc_pfp_shower_energy', 'pfp_idx', 'slc_pfp_shower_dir_x', 'slc_pfp_shower_dir_y', 'slc_pfp_shower_dir_z'])
    shw1 = shw1.merge(temp, how='inner', on=['run','subrun','event','slc_idx'])
    
    #2+ shower selection
    shw2more = concat[concat['n_pfp'] >= 2]
    temp = temp.drop(columns = ['slc_pfp_shower_energy', 'pfp_idx', 'slc_pfp_shower_dir_x', 'slc_pfp_shower_dir_y', 'slc_pfp_shower_dir_z'])
    shw2more = shw2more.merge(temp, how='inner', on=['run','subrun','event','slc_idx'])
    
    return shw1, shw2more

In [ ]:
df_hnl_shw1, df_hnl_shw2 = split_my_df(df_hnl)

df_nu_shw1, df_nu_shw2 = split_my_df(df_nu)

#Nothing is left in Intime cosmics
#df_cos_shw1, df_cos_shw2 = split_my_df(df_cos)

<h3>Single Shower Energy</h3>

In [ ]:
plot_slc_var(df_hnl_shw1, df_nu_shw1, df_cos,
#plot_slc_var(df_hnl_shw1, df_nu_shw1, df_cos_shw1,
                    true_counts, start_counts, 
                    'slc_pfp_shower_energy', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 3000, xnbin = 50,
                    xtitle = 'Slice With 1 Shower Energy [MeV]'
            )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"single_shower_energy.png", dpi = 200)
#-----------------------------------#
plt.show()

<h3>Apply Cut</h3>

<h3>Two or More Shower</h3>

In [ ]:
def pi0mass(x):
    
    mBest = -999
    diff = 99999;
    
    for i in range(0, x['n_pfp']):
        for j in range(0, x['n_pfp']):
            if i == j:
                continue
            else:
                v1 = [x['slc_pfp_shower_dir_x'][0], x['slc_pfp_shower_dir_y'][0], x['slc_pfp_shower_dir_z'][0]]
                v2 = [x['slc_pfp_shower_dir_x'][1], x['slc_pfp_shower_dir_y'][1], x['slc_pfp_shower_dir_z'][1]]
    
                v1_u = unit_vector(v1)
                v2_u = unit_vector(v2)
    
                cosTheta = np.clip(np.dot(v1_u, v2_u), -1.0, 1.0)

                mass = np.sqrt(2 * x['slc_pfp_shower_energy'][0] * x['slc_pfp_shower_energy'][1] * (1 - cosTheta ));
                
                if abs(mass - 135) < diff:
                    mBest = mass
    return mBest

In [ ]:
df_hnl_shw2['pi0_mass'] = df_hnl_shw2.apply(lambda row: pi0mass(row), axis = 1)

#df_nu_shw2['pi0_mass'] = ""
df_nu_shw2['pi0_mass'] = df_nu_shw2.apply(lambda row: pi0mass(row), axis = 1)

df_cos['pi0_mass']  = ""
#df_cos_shw2['pi0_mass'] = df_cos_shw2.apply(lambda row: pi0mass(row), axis = 1)

In [ ]:
plot_slc_var(df_hnl_shw2, df_nu_shw2, df_cos,
#plot_slc_var(df_hnl_shw2, df_nu_shw2, df_cos_shw2,
                    true_counts, start_counts, 
                    'pi0_mass', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 600, xnbin = 50,
                    xtitle = 'Pi 0 Mass [MeV]'
            )
# only one line may be specified; full height
plt.axvline(x = 135, color = col_dict['Aqua'], linewidth = 3)

#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"pizero_mass_precut.png", dpi = 200)
#-----------------------------------#

plt.show()

<h3>Apply Cut</h3>

In [ ]:
df_hnl_shw2 = df_hnl_shw2[(df_hnl_shw2['pi0_mass'] > 20) &(df_hnl_shw2['pi0_mass'] < 250)]
df_nu_shw2 = df_nu_shw2[(df_nu_shw2['pi0_mass'] > 20) &(df_nu_shw2['pi0_mass'] < 250)]
#df_cos_shw2 = df_cos_shw2[(df_cos_shw2['pi0_mass'] > 20) &(df_cos_shw2['pi0_mass'] < 250)]

In [ ]:
plot_slc_var(df_hnl_shw2, df_nu_shw2, df_cos,
#plot_slc_var(df_hnl_shw2, df_nu_shw2, df_cos_shw2,
                    true_counts, start_counts, 
                    'pi0_mass', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 600, xnbin = 50,
                    xtitle = 'Pi 0 Mass [MeV]'
            )
# only one line may be specified; full height
plt.axvline(x = 135, color = col_dict['Aqua'], linewidth = 3)
 
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"pizero_mass_postcut.png", dpi = 200)
#-----------------------------------#

plt.show()

<h3>Merge The Cut Back</h3>

In [ ]:
def apply_shower_cut(df, dfshw1, dfshw2):
    temp1 = dfshw1[['run','subrun','event','slc_idx']]
    temp1 = temp1.merge(df, how='inner', on=['run','subrun','event','slc_idx'])
    
    temp2 = dfshw2[['run','subrun','event','slc_idx']]
    temp2 = temp2.merge(df, how='inner', on=['run','subrun','event','slc_idx'])
    
    concat = pd.concat([temp1, temp2])
    concat = concat.drop_duplicates()
    
    return concat

In [ ]:
df_hnl = apply_shower_cut(df_hnl, df_hnl_shw1, df_hnl_shw2)
df_nu = apply_shower_cut(df_nu, df_nu_shw1, df_nu_shw2)
#df_cos = apply_shower_cut(df_cos, df_cos_shw1, df_cos_shw2)

<h3>Save Purity/Eff</h3>

In [ ]:
purity, eff = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)

purity_arr.append(purity)
eff_arr.append(eff)
cut_arr.append("Pion Zero Mass")

print(purity_arr)
print(eff_arr)
print(cut_arr)

<h1> Make Beam Bucket - Post PID</h1>

In [ ]:
hist, bins = plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'mod_t', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 19, xnbin = 19,
                    xtitle = 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]'
                    )

#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"beam_bucket_postpid.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1>Between Bucket Cut</h1>

In [ ]:
def cutModt(df, score):
    when_modt = df["mod_t"] >= score
    df = df[when_modt]
    return df    

<h3>Apply Cut</h3>

In [ ]:
df_hnl = cutModt(df_hnl, 10)
df_nu = cutModt(df_nu, 10)
df_cos = cutModt(df_cos, 10)

<h3>Save Purity/Eff</h3>

In [ ]:
purity, eff = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)

purity_arr.append(purity)
eff_arr.append(eff)
cut_arr.append("Between Bucket")

print(purity_arr)
print(eff_arr)
print(cut_arr)

In [ ]:
hist, bins = plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'mod_t', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 19, xnbin = 19,
                    xtitle = 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]'
                    )

#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"beam_bucket_final.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1>Plot Purity/Efficiency </h1>

In [ ]:
print(eff_arr)
print(purity_arr)
print(cut_arr)

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(10, 6), sharex = True)
ax2 = ax1.twinx()

t = np.arange(0, len(cut_arr) , 1)

effcol = col_dict['MintGreen']
pcol = col_dict['Flamingo']

ax1.plot(t, eff_arr, c = effcol, marker = 'o', markersize = 5, ls='--')
ax2.plot(t, purity_arr, c = pcol, marker = 'o', markersize = 5, ls='--')

ax1.set_ylim(-5,105)
ax2.set_ylim(-5,105)

ax1.grid(color='gainsboro', which='both', axis='both', linestyle='--')

ax1.set_ylabel("Selection Effciency [%]", c = effcol, fontsize = 16)
ax2.set_ylabel("Purity [%]", c = pcol, fontsize = 16)

plt.xticks(range(0, len(t)))
ax1.set_xticklabels(cut_arr, rotation = 45, ha="right", fontsize = 16)

ax1.tick_params(bottom = True, top = True, left = True, right = False)
ax2.tick_params(bottom = True, top = True, left = False, right = True)

ax1.tick_params(axis = 'x', labelsize = 16, direction = 'in')
ax1.tick_params(axis = 'y', labelsize = 16, direction = 'in')
ax2.tick_params(axis = 'y', labelsize = 16, direction = 'in')

fig.tight_layout()

#-----------------------------------#
if ifSave:
    if ifStrict:
        plt.savefig(savePath+"peff_strict.png", dpi = 200)
    else:
        plt.savefig(savePath+"peff_relaxed.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1> Save Cut Dataframe to Pickle</h1>

In [ ]:
df_hnl.to_pickle("./df_hnl.pkl", protocol = 5)

In [ ]:
df_nu.to_pickle("./df_nu.pkl", protocol = 5)

In [ ]:
df_cos.to_pickle("./df_cos.pkl", protocol = 5)